Формулировка задания
--------------------
Имеется перехваченный файл, зашифрованный симметричным алгоритмом шифрования *AES-256*, вместе с файлом передавались данные для его расшифровки: его ключ и вектор инициализации, известно, что они передавались не в открытом виде, а также были зашифрованы ассиметричным шифром - *RSA*.

* Key:

```WRGGNuGS9ykJNTvllzUyJ89D7128Up/J/K+GSU07w3mmSGNRNRTBt+PcKC5V7rFsbje0c6g0CvFSFIHAAQeOOMZYMaqDmGTfMJUG5Ir4Uf0oeF5G5u0BMKp4HGpFDHwAhFhJl0Y2ZerhNXPMo8E4dX0VZKhxjAvUuIFmgk4dzcE=```

* IV:

```hJN7pjqF1t6Oh7MG6eT48DI7Sv/Ca2CO8e/CJcPw367Bem0U52omnq2bkqVzVI1BzYZ9B7LE4G91ddlOzXKJSAxUdCNrlASqoG57GG1MgBJyq48qFaqDxFYBHo8IHEy2e68FSzQ5/5as5S3tf+/FCWl+u80GLl1eqGnDR/BJkmc=```

Эти данные нужны принимающей стороне, для расшифровки контента. Разберемся со значением IV. Интернет говорит нам, что IV –это вектор инициализации, подаваемый на вход в таких режимах шифрования, как CBC, CFB и OFB. Причем для обычного открытого текста можно использовать CBC, CFB или OFB. Для шифрования файлов лучше пользоваться CBC: значительно увеличивается безопасность, при появлении ошибок в хранимых данных почти никогда не бывает сбоев синхронизации. [Побробнее можно почитать здесь.](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%B6%D0%B8%D0%BC_%D1%88%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F#Initialization_vector_(IV))

При попытке воспроизвести видео видидим ошибку...

!['Ошибка при воспроизведении'](figures/error.png)

На этом наши данные не заканчиваются, удалось добыть базу данных открытых ключей RSA (к сожалению, без закрытого, но это поправимо). Распаковываем архив: 

!['Содержимое архива'](figures/archive_content.png)

Получаем множество .pem файлов. В них хранятся открытые ключи RSA, выглядят они следующим образом:

!['Открытый ключ'](figures/pemfile.png)


Атака на RSA
------------
Предположим, что исходный ключ симметричного шифрования был зашифрован одним из этих открытых ключей. Чтобы его расшифровать, нам необходимо получить закрытый ключ.

[Гуглим атаки на RSA](https://www.sjoerdlangkemper.nl/2019/06/19/attacking-rsa/). Когда в распоряжении есть несколько открытых ключей, можно предположить, что некоторые из них являются слабыми. Это означает, что они имеют общие значения $p$ или $q$. Узнав разложение $n$ на множители, может узнать закрытый ключ для данного открытого ключа. [Освежим в памяти наши знания об RSA и еще раз перечитаем предыдущее предложение](https://en.wikipedia.org/wiki/RSA_(cryptosystem))... Отлично, теперь попробуем найти слабые ключи.

Поиск слабых ключей
-------------------
Пробуем найти наибольшие общие делители у ключей с помощью алгоритма [Евклида (вдруг кто забыл)](https://en.wikipedia.org/wiki/Euclidean_algorithm). Если НОД каких-либо ключей не равен 1, это означает, что $n$, соответствующие ключам делятся на НОД. Делим $n$ на НОД и находим искомое разложение. Скрипт для нахождения слабых ключей:


In [2]:
import rsa #импортируем нужные модули
import base64
import math
key_list=[] #создаем список, в котором будут хранится ключи
for i in range(1, 101): #циклом проходимся по каждому ключу
    path = 'challenge\\'
    path += str(i)
    path += '.pem'
    with open(path, 'rb') as f:
        public_file = f.read() #читаем файл
    pubkey = rsa.PublicKey.load_pkcs1_openssl_pem(public_file) #загружаем открытый ключ из файла, он содержит параметры n и e
    key_list.append(pubkey) #добавляем ключ в список
    #print(pubkey.n)

for i in range (len(key_list)-1):#проходимся по спику открытых ключей
    for j in range (i+1,len(key_list)):
        g=math.gcd(key_list[i].n,key_list[j].n) #для всех ключей ищем наибольший общий делитель
        if g!=1: #вывходим номера ключей и их НОД, если их значения n не взаимно простые
            print(i, j)
            print(g)

6 92
11714776996979588435440066274186746431207255454202665944875994417925671637361979718990753187254963362590523267356636189119746530253241195041886537439669297
8 43
12746114090037338638405558348364753170762024925046410448750084084398232130814610810740410270513256665449277121134270816451924978100353060942631979836542387
28 81
13255021146474500311567608978490890706169542435903223652857277300729054042351345458519493118521383085015716518452176586805584419240894763844777441159376627
33 79
13307692754962545190733618818859249009308397605584526413665519201272482870171338937280605559186730933842250237720203697942551335313430434560173585438909597
57 70
12913750264623462276337425644128711174675813336880465068461018681645080047918835348323621347173714301224881492646558739535118834131362641424915974945262413
59 96
13223199974589313585283471854827363329451685303943205044027865131047695079691108706585803904203301545648101019060082323960067905432343412585098414381558929


Восстановление закрытого ключа
------------------------------
Восстановим закрытый ключ, например, для 6 открытого ключа. Значение закрытого ключа получаем путем нахождения обратного к числу $e$ по модулю $φ(n)$, где $$φ(n) = (p − 1)(q − 1)$$
$$d=e^{-1}(mod φ(n))$$  


In [5]:
weak1=6
weak2=92
e=key_list[weak1].e
print("e:",e)
p=math.gcd(key_list[weak1].n, key_list[weak2].n) #НОД значений n слабых ключей
print("p:",p)
n=key_list[weak1].n
print("n:",n)
q=n//p #Находим q
print("q:",q)
phi=(p-1)*(q-1) #теперь можем найти phi
print("phi:",phi)
d=pow(e,-1,phi) #находим закрытый ключ обратное мультипликативное к e по модулю phi
print("d:",d)
mykey = rsa.PrivateKey(n, e, d, p, q)
print(mykey.d)


e: 65537
p: 11714776996979588435440066274186746431207255454202665944875994417925671637361979718990753187254963362590523267356636189119746530253241195041886537439669297
n: 136269636317215868658126726142543242028128679787201513621377420299644359247151157885793577216543689892988935986714087409150506883630386841292060595217129497897100280678153687017820663980404875865314501020301179267627899307057160787226214936662085381326053730017478234531591680965138499420169342895677786825703
q: 11632285988230946246714551486716113190291520068494163099500564698778303325737351498239617583337540511764131867572374744310703041201014109767591825596334999
phi: 136269636317215868658126726142543242028128679787201513621377420299644359247151157885793577216543689892988935986714087409150506883630386841292060595217129474550037295467619004863202903077545254366538978323472134891068782603082197687894997706291314788822179375362343305520658250515567045164864533417314750821408
d: 575959980772217153948168258258456719742918

Расшифровка ключа AES и IV
--------------------------
Ключ и вектор инициализации хранятся в в файлах key.txt и iv.txt соответственно. Теперь попробуем с помощью закрытого ключа расшифровать наш симметричный ключ. Обращаем внимание, что байты ключа и вектора инициализации закодированы в формате base64, то есть перед их расшифровкой с помощью закрытого ключа необходима предварительная обработка.

In [6]:
f = open('key.txt',mode='r')
base64_message =f.read()
base64_bytes = base64_message.encode('utf-8')
message_bytes = base64.b64decode(base64_bytes)
print(rsa.decrypt(message_bytes,mykey))

f = open('iv.txt',mode='r')
base64_message =f.read()
base64_bytes = base64_message.encode('utf-8')
message_bytes = base64.b64decode(base64_bytes)
print(rsa.decrypt(message_bytes,mykey))


b'E8B6C00C9ADC5E75BB656ECD429CB1643A25B111FCD22C6622D53E0722439993'
b'E486BB61EB213ED88CC3CFB938CD58D7'


Расшифровка файла
-----------------
Отлично, теперь у нас есть все данные для того, чтобы расшифровать исходный файл. Воспользуемся утилитой openssl для расшифровки:

```openssl enc -nosalt -aes-256-cbc -d -in файлсмерти.mp4 -out файлсмертиdecrypted.mp4 -base64 -K E8B6C00C9ADC5E75BB656ECD429CB1643A25B111FCD22C6622D53E0722439993 -iv E486BB61EB213ED88CC3CFB938CD58D7```

Получаем расшифрованный файл и наслаждаемся просмотром:

!['Расшифрованное изображение'](figures/decrypted.png)
